Script Goals:
Based on FloatCropperLipstick.ipynb
1. Generate a bunch of .png files to crop all at once. 
2. Save the green output and .npy files all at once

# **Step 1:** Find all of the .float, output.png, GreenOutput.png, and .npy filepaths

In [18]:
num_images = 15
base_name = 'CT30_'
float_directory = '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/'
crop_directory = '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Sunscreen Input/Lower Leg/'
PNGpaths = []
Floatpaths = []
Greenpaths = []
HDRpaths = []
NPYpaths = []
for i in range(num_images):
    Floatpaths.append(float_directory + base_name + str(i+1) + '.float')
    HDRpaths.append(float_directory + base_name + str(i+1) + '.hdr')
    PNGpaths.append(crop_directory + base_name + str(i+1) + 'output.png')
    Greenpaths.append(crop_directory + base_name + str(i+1) + 'GreenOutput.png')
    NPYpaths.append(crop_directory + base_name + str(i+1) + 'Sunscreen.npy')
print(Floatpaths)
print(HDRpaths)
print(Greenpaths)
print(PNGpaths)
print(NPYpaths)

['/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_1.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_2.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_3.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_4.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_5.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_6.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_7.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_8.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Images/Sunscreen 1/CT30_9.float', '/Users/maycaj/Documents/Hyperspectral-Imaging_II/Lipstick_Sunscreen Ima

# **Step 2:** Generate all of the Output.png files from the .float files

# **Step3:** Use the output.png files to generate the .npy files 

In [19]:
import numpy as np
import spectral.io.envi as envi
from PIL import Image

# Define the green color you want to use
green = (0, 255, 0, 255)  # RGBA format (R, G, B, A)

for i in range(num_images):
    image = Image.open(PNGpaths[i]) #open PNG
    PNG_NP = image.load() #convert PNG to npy
    x_indexes = []
    y_indexes = []
    for x in range(image.width):
        for y in range(image.height):
            r, g, b, a = PNG_NP[x, y]
            if a != 0 or (r != 0 and g != 0 and b != 0):
                x_indexes.append(x) # pull indecies where there are crops
                y_indexes.append(y)
    #find the x and y coordinates of the crop
    x_range = [np.min(x_indexes), np.max(x_indexes)] 
    y_range = [np.min(y_indexes), np.max(y_indexes)]

    # load the .float file
    img = envi.open(HDRpaths[i],Floatpaths[i])
    # crop the image
    cropped = img[y_range[0]:y_range[1], x_range[0]:x_range[1],:]
    # save the .npy
    np.save(NPYpaths[i], cropped)
    # Ensure saved data is correct type
    loaded = np.load(NPYpaths[i])
    if loaded.dtype == 'float32':
        print('pixel_loaded has correct datatype')
    else:
        raise Exception('INCORRECT DATATYPE')


pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype
pixel_loaded has correct datatype


# **Step 4:** Explore the cropped data

In [24]:
import matplotlib.pyplot as plt
from ipywidgets import interact

# choose which image to display
i = 14
loaded = np.load(NPYpaths[i])
# fig, ax = plt.subplots()
# ax.imshow(loaded[:, :, 49:50], cmap='gray')
# ax.set_title('Gray Image – Masked')

def interactive_plot(band):
    # Create a new figure with two subplots
    fig, (ax5, ax4) = plt.subplots(1, 2)

    # Only display the 49th band in gray
    grey_band = loaded[:, :, band:(band + 1)]
    print('Grey_bands shape:', grey_band.shape)

    # Display the gray image in the first subplot
    ax5.imshow(grey_band, cmap='gray')
    ax5.set_title(f'Gray Image – cropped (band = {band})')

    # Grab the colored bands
    loaded_red = loaded[:,:,20]
    loaded_green = loaded[:,:,35]
    loaded_blue = loaded[:,:,60]
   
    loaded_color = np.dstack((loaded_blue, loaded_green, loaded_red))

    # Display the color image in the second subplot
    ax4.imshow(loaded_color)
    ax4.set_title(f'Color Image number {i+1}')

    # Show the figure with both subplots
    plt.show()

interact(interactive_plot, band=(0, 127, 1))

interactive(children=(IntSlider(value=63, description='band', max=127), Output()), _dom_classes=('widget-inter…

<function __main__.interactive_plot(band)>